### Notebook setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Create Engine/Session

In [2]:
import sqlalchemy as sa
import sqlalchemy.orm

engine = sa.create_engine(
    "cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable", echo=True
)
engine

Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable)

<IPython.core.display.Javascript object>

In [3]:
LocalSession = sa.orm.sessionmaker(engine)
LocalSession

sessionmaker(class_='Session', bind=Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable), autoflush=True, autocommit=False, expire_on_commit=True)

<IPython.core.display.Javascript object>

### Cockroachdb cluster id

In [4]:
with LocalSession() as session:
    statement = sa.text(
        "SELECT value FROM crdb_internal.node_build_info WHERE field = 'ClusterID';"
    )
    result = session.execute(statement)

result.first()

2021-10-19 13:11:09,709 INFO sqlalchemy.engine.Engine select current_schema()
2021-10-19 13:11:09,710 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-10-19 13:11:09,713 INFO sqlalchemy.engine.Engine select version()
2021-10-19 13:11:09,714 INFO sqlalchemy.engine.Engine [generated in 0.00122s] {}
2021-10-19 13:11:09,803 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%(val)s)
2021-10-19 13:11:09,803 INFO sqlalchemy.engine.Engine [generated in 0.00090s] {'val': 'sqlalchemy-cockroachdb 1.4.1'}
2021-10-19 13:11:09,805 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%(val)s)
2021-10-19 13:11:09,806 INFO sqlalchemy.engine.Engine [cached since 0.003056s ago] {'val': 'sqlalchemy 1.4.25'}
2021-10-19 13:11:09,808 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:09,809 INFO sqlalchemy.engine.Engine SELECT value FROM crdb_internal.node_build_info WHERE field = 'ClusterID';
2021-10-19 13:11:09,810 INFO sqlalchemy.engine.Engine

('fbe0e6ed-49b9-4f47-a7ec-3f4ee14e978d',)

<IPython.core.display.Javascript object>

### Drop/Create tables

In [5]:
from models import Base, User, Message

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

2021-10-19 13:11:10,402 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:10,403 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%(schema)s
2021-10-19 13:11:10,404 INFO sqlalchemy.engine.Engine [generated in 0.00102s] {'schema': 'public'}
2021-10-19 13:11:10,412 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%(schema)s
2021-10-19 13:11:10,413 INFO sqlalchemy.engine.Engine [cached since 0.009945s ago] {'schema': 'public'}
2021-10-19 13:11:10,417 INFO sqlalchemy.engine.Engine COMMIT
2021-10-19 13:11:10,418 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:10,419 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%(schema)s
2021-10-19 13:11:10,421 INFO sqlalchemy.engine.Engine [cached since 0.01737s ago] {'schema': 'public'}
2021-10-19 13:11:10,428 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_sch

<IPython.core.display.Javascript object>

### Add data

Nick, Eli, and Paul sing 99 bottles of beer...

In [6]:
users = [User(name="Nick"), User(name="Eli"), User(name="Paul")]
users

[User(id=None, name='Nick'),
 User(id=None, name='Eli'),
 User(id=None, name='Paul')]

<IPython.core.display.Javascript object>

In [7]:
with LocalSession() as session:
    session.add_all(users)
    session.commit()

2021-10-19 13:11:17,697 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:17,700 INFO sqlalchemy.engine.Engine INSERT INTO "user" (name) VALUES (%(name)s) RETURNING "user".id
2021-10-19 13:11:17,701 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ({'name': 'Nick'}, {'name': 'Eli'}, {'name': 'Paul'})
2021-10-19 13:11:19,779 INFO sqlalchemy.engine.Engine COMMIT


<IPython.core.display.Javascript object>

In [8]:
import itertools

cycle = itertools.cycle(users)
messages = []

for i in reversed(range(100)):
    content = f"{i} bottles of beer on the wall..."
    user = next(cycle)
    message = Message(user=user, content=content)
    messages.append(message)

with LocalSession() as session:
    session.add_all(messages)
    session.commit()

2021-10-19 13:11:20,516 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:20,520 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-10-19 13:11:20,521 INFO sqlalchemy.engine.Engine [generated in 0.00136s] {'pk_1': 703131417382486017}
2021-10-19 13:11:20,856 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-10-19 13:11:20,857 INFO sqlalchemy.engine.Engine [cached since 0.3367s ago] {'pk_1': 703131417382617089}
2021-10-19 13:11:20,860 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %(pk_1)s
2021-10-19 13:11:20,861 INFO sqlalchemy.engine.Engine [cached since 0.3411s ago] {'pk_1': 703131417382649857}
2021-10-19 13:11:20,868 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%(content)s, %(user_id)s) RETURNING message.id
2021-10

<IPython.core.display.Javascript object>

### Query for data

#### Count query

In [9]:
### Expect to see 3 users here

with LocalSession() as session:
    statement = sa.select(sa.func.count(User.id))
    results = session.scalar(statement)

results

2021-10-19 13:11:22,688 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:22,692 INFO sqlalchemy.engine.Engine SELECT count("user".id) AS count_1 
FROM "user"
2021-10-19 13:11:22,693 INFO sqlalchemy.engine.Engine [generated in 0.00179s] {}
2021-10-19 13:11:22,697 INFO sqlalchemy.engine.Engine ROLLBACK


3

<IPython.core.display.Javascript object>

In [10]:
### Expect to see 100 messages here

with LocalSession() as session:
    statement = sa.select(sa.func.count(Message.id))
    results = session.scalar(statement)

results

2021-10-19 13:11:23,588 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:23,590 INFO sqlalchemy.engine.Engine SELECT count(message.id) AS count_1 
FROM message
2021-10-19 13:11:23,591 INFO sqlalchemy.engine.Engine [generated in 0.00093s] {}
2021-10-19 13:11:23,594 INFO sqlalchemy.engine.Engine ROLLBACK


100

<IPython.core.display.Javascript object>

#### Query users with messages

In [11]:
with LocalSession() as session:
    statement = sa.select(User).options(sa.orm.joinedload(User.messages))
    results = session.execute(statement)

results

2021-10-19 13:11:24,355 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:24,358 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name, message_1.id AS id_1, message_1.content, message_1.user_id 
FROM "user" LEFT OUTER JOIN message AS message_1 ON "user".id = message_1.user_id
2021-10-19 13:11:24,359 INFO sqlalchemy.engine.Engine [generated in 0.00078s] {}
2021-10-19 13:11:24,364 INFO sqlalchemy.engine.Engine ROLLBACK


<IPython.core.display.Javascript object>

In [12]:
users = results.unique().scalars().all()
users

[User(name='Nick', id=703131417382486017, messages=[Message(user_id=703131417382486017, content='99 bottles of beer on the wall...', id=703131424377700353), Message(user_id=703131417382486017, content='0 bottles of beer on the wall...', id=703131424381075457), Message(user_id=703131417382486017, content='3 bottles of beer on the wall...', id=703131424380977153), Message(user_id=703131417382486017, content='6 bottles of beer on the wall...', id=703131424380878849), Message(user_id=703131417382486017, content='9 bottles of beer on the wall...', id=703131424380780545), Message(user_id=703131417382486017, content='12 bottles of beer on the wall...', id=703131424380682241), Message(user_id=703131417382486017, content='15 bottles of beer on the wall...', id=703131424380583937), Message(user_id=703131417382486017, content='18 bottles of beer on the wall...', id=703131424380485633), Message(user_id=703131417382486017, content='21 bottles of beer on the wall...', id=703131424380387329), Message

<IPython.core.display.Javascript object>

In [13]:
users[0].name

'Nick'

<IPython.core.display.Javascript object>

In [14]:
users[0].messages[:5]

[Message(user_id=703131417382486017, content='99 bottles of beer on the wall...', id=703131424377700353),
 Message(user_id=703131417382486017, content='0 bottles of beer on the wall...', id=703131424381075457),
 Message(user_id=703131417382486017, content='3 bottles of beer on the wall...', id=703131424380977153),
 Message(user_id=703131417382486017, content='6 bottles of beer on the wall...', id=703131424380878849),
 Message(user_id=703131417382486017, content='9 bottles of beer on the wall...', id=703131424380780545)]

<IPython.core.display.Javascript object>

In [15]:
for m in users[0].messages[:5]:
    print(m.content)

99 bottles of beer on the wall...
0 bottles of beer on the wall...
3 bottles of beer on the wall...
6 bottles of beer on the wall...
9 bottles of beer on the wall...


<IPython.core.display.Javascript object>

#### Chained statement example

In [16]:
with LocalSession() as session:
    statement = sa.select(User)
    statement = statement.where(User.name == "Eli")
    statement = statement.options(sa.orm.selectinload(User.messages))
    results = session.execute(statement)

user = results.scalars().first()
user

2021-10-19 13:11:27,231 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:27,236 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name 
FROM "user" 
WHERE "user".name = %(name_1)s
2021-10-19 13:11:27,238 INFO sqlalchemy.engine.Engine [generated in 0.00175s] {'name_1': 'Eli'}
2021-10-19 13:11:27,245 INFO sqlalchemy.engine.Engine ROLLBACK
2021-10-19 13:11:27,251 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:27,259 INFO sqlalchemy.engine.Engine SELECT message.user_id AS message_user_id, message.id AS message_id, message.content AS message_content 
FROM message 
WHERE message.user_id IN (%(primary_keys_1)s)
2021-10-19 13:11:27,262 INFO sqlalchemy.engine.Engine [generated in 0.00427s] {'primary_keys_1': 703131417382617089}


User(name='Eli', id=703131417382617089, messages=[Message(user_id=703131417382617089, content='98 bottles of beer on the wall...', id=703131424377831425), Message(user_id=703131417382617089, content='95 bottles of beer on the wall...', id=703131424377962497), Message(user_id=703131417382617089, content='92 bottles of beer on the wall...', id=703131424378060801), Message(user_id=703131417382617089, content='89 bottles of beer on the wall...', id=703131424378159105), Message(user_id=703131417382617089, content='86 bottles of beer on the wall...', id=703131424378257409), Message(user_id=703131417382617089, content='83 bottles of beer on the wall...', id=703131424378355713), Message(user_id=703131417382617089, content='80 bottles of beer on the wall...', id=703131424378454017), Message(user_id=703131417382617089, content='77 bottles of beer on the wall...', id=703131424378552321), Message(user_id=703131417382617089, content='74 bottles of beer on the wall...', id=703131424378650625), Messa

<IPython.core.display.Javascript object>

In [17]:
user.name

'Eli'

<IPython.core.display.Javascript object>

In [18]:
for m in user.messages[:5]:
    print(m.content)

98 bottles of beer on the wall...
95 bottles of beer on the wall...
92 bottles of beer on the wall...
89 bottles of beer on the wall...
86 bottles of beer on the wall...


<IPython.core.display.Javascript object>

### Using classmethod queries

In [19]:
with LocalSession() as session:
    messages = Message.from_user(session, "Eli")

len(messages)

2021-10-19 13:11:28,990 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-19 13:11:28,992 INFO sqlalchemy.engine.Engine SELECT message.id, message.content, message.user_id 
FROM message JOIN "user" ON "user".id = message.user_id 
WHERE "user".name = %(name_1)s
2021-10-19 13:11:28,993 INFO sqlalchemy.engine.Engine [generated in 0.00089s] {'name_1': 'Eli'}
2021-10-19 13:11:28,998 INFO sqlalchemy.engine.Engine ROLLBACK


33

<IPython.core.display.Javascript object>

In [20]:
messages[:5]

[Message(user_id=703131417382617089, content='98 bottles of beer on the wall...', id=703131424377831425),
 Message(user_id=703131417382617089, content='95 bottles of beer on the wall...', id=703131424377962497),
 Message(user_id=703131417382617089, content='92 bottles of beer on the wall...', id=703131424378060801),
 Message(user_id=703131417382617089, content='89 bottles of beer on the wall...', id=703131424378159105),
 Message(user_id=703131417382617089, content='86 bottles of beer on the wall...', id=703131424378257409)]

<IPython.core.display.Javascript object>